# Fast Color Transfer using Mean and Standard Deviation


#### Imports


In [ ]:
import os
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install opencv-python
!{sys.executable} -m pip install matplotlib

import numpy as np
import cv2
import matplotlib.pyplot as plt

#### Google Drive vs Local Execution


In [ ]:
# If running on a local machine it is recommended to start a virtual environment
# before running this notebook.
try:
    from google.colab import drive
    IN_COLAB = True
    print("Running in Google Colab")
except:
    IN_COLAB = False
    print("Running locally")

if IN_COLAB:
    drive.mount('/content/drive')
    # Edit base path as needed
    base_path = '/content/drive/My Drive/Colab Notebooks/Color Transfer'
else:
    # Edit base path as needed
    base_path = os.getcwd()

print(f"Base path: {base_path}")
image_path = base_path + '/images'
print(f"Image path: {image_path}")
os.chdir(image_path)

In [ ]:
# Display image filenames for sanity's sake
print("Images")
os.listdir(image_path)

#### Function Definitions for Color Transfer


![formula](attachments/formula.png)


In [ ]:
def color_transfer(reference, input):
    # OpenCV expects floats to be 32-bit.
    # Convert the image to CIE L*a*b color space
    # Note: images imported by using cv2.imread() are imported as BGR rather
    # than RBG. It is recommended to work with BGR inside of opencv and to
    # convert to RGB only when needed.
    reference = cv2.cvtColor(reference, cv2.COLOR_BGR2LAB).astype("float32")
    input = cv2.cvtColor(input, cv2.COLOR_BGR2LAB).astype("float32")

    # Split images into L*a*b color space
    (l_ref, a_ref, b_ref) = cv2.split(reference)
    (l_in, a_in, b_in) = cv2.split(input)

    # Apply transform described in the paper
    # Refer to formula above
    l_out = rescale(
        ((l_ref.std() / l_in.std()) * (l_in - l_in.mean())) + l_ref.mean())
    a_out = rescale(
        ((a_ref.std() / a_in.std()) * (a_in - a_in.mean())) + a_ref.mean())
    b_out = rescale(
        ((b_ref.std() / b_in.std()) * (b_in - b_in.mean())) + b_ref.mean())

    # Generate output image
    output = cv2.merge([l_out, a_out, b_out])
    output = cv2.cvtColor(output.astype("uint8"), cv2.COLOR_LAB2BGR)

    # Show Histogram if needed
    # show_hist(l_in, a_in, b_in, l_ref, a_ref, b_ref, l_out, a_out, b_out)

    # Return the color transferred image
    return output


def rescale(arr):
    arr_min, arr_max = arr.min(), arr.max()
    new_min, new_max = max(arr_min, 0), min(arr_max, 255)

    # If the array is out of bounds of 0 to 255
    # scale it back into the range of 0 to 255
    scaled = new_min + (new_max - new_min) * \
        (arr - arr_min) / (arr_max - arr_min)
    return scaled

#### Function Definitions to show images


In [ ]:
def resize_image(image, width=700):
    # Resize the image to have a constant width
    r = width / float(image.shape[1])
    dim = (width, int(image.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)

    # Return the resized image in RGB format for matplotlib
    return cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)


def show_hist(l_in, a_in, b_in, l_ref, a_ref, b_ref, l_out, a_out, b_out):
    # Calculate histograms with 64 bins
    hist_l_in = cv2.calcHist([l_in], [0], None, [64], [0, 256])
    hist_a_in = cv2.calcHist([a_in], [0], None, [64], [0, 256])
    hist_b_in = cv2.calcHist([b_in], [0], None, [64], [0, 256])

    hist_l_ref = cv2.calcHist([l_ref], [0], None, [64], [0, 256])
    hist_a_ref = cv2.calcHist([a_ref], [0], None, [64], [0, 256])
    hist_b_ref = cv2.calcHist([b_ref], [0], None, [64], [0, 256])

    hist_l_out = cv2.calcHist([l_out], [0], None, [64], [0, 256])
    hist_a_out = cv2.calcHist([a_out], [0], None, [64], [0, 256])
    hist_b_out = cv2.calcHist([b_out], [0], None, [64], [0, 256])

    # Normalize histograms for better visualization
    hist_l_in = hist_l_in / hist_l_in.sum()
    hist_a_in = hist_a_in / hist_a_in.sum()
    hist_b_in = hist_b_in / hist_b_in.sum()

    hist_l_ref = hist_l_ref / hist_l_ref.sum()
    hist_a_ref = hist_a_ref / hist_a_ref.sum()
    hist_b_ref = hist_b_ref / hist_b_ref.sum()

    hist_l_out = hist_l_out / hist_l_out.sum()
    hist_a_out = hist_a_out / hist_a_out.sum()
    hist_b_out = hist_b_out / hist_b_out.sum()

    # Create x-axis values (pixel values)
    bins = np.arange(64)

    # Create a figure and axis for each image
    fig, axs = plt.subplots(1, 3, figsize=(22, 4))

    # Display the histograms side by side
    axs[0].bar(bins, hist_l_in.flatten(), color='red', alpha=0.7, label='L')
    axs[0].bar(bins, hist_a_in.flatten(), color='green', alpha=0.7, label='A')
    axs[0].bar(bins, hist_b_in.flatten(), color='blue', alpha=0.7, label='B')
    axs[0].legend()
    axs[0].axis('off')

    axs[1].bar(bins, hist_l_ref.flatten(), color='red', alpha=0.7, label='L')
    axs[1].bar(bins, hist_a_ref.flatten(), color='green', alpha=0.7, label='A')
    axs[1].bar(bins, hist_b_ref.flatten(), color='blue', alpha=0.7, label='B')
    axs[1].legend()
    axs[1].axis('off')

    axs[2].bar(bins, hist_l_out.flatten(), color='red', alpha=0.7, label='L')
    axs[2].bar(bins, hist_a_out.flatten(), color='green', alpha=0.7, label='A')
    axs[2].bar(bins, hist_b_out.flatten(), color='blue', alpha=0.7, label='B')
    axs[2].legend()
    axs[2].axis('off')

    plt.show()


def showcase(reference, input):
    # Generate a transferred image
    output = color_transfer(reference, input)

    # Create a figure and axis for each image
    fig, axs = plt.subplots(1, 3, figsize=(22, 9))

    # Display the images side by side
    axs[0].imshow(resize_image(input))
    axs[0].axis('off')
    axs[0].set_title('Input Image')

    axs[1].imshow(resize_image(reference))
    axs[1].axis('off')
    axs[1].set_title('Reference Palette')

    axs[2].imshow(resize_image(output))
    axs[2].axis('off')
    axs[2].set_title('Output Image')

    plt.show()

#### Showcase


In [ ]:
# Showcase
selection = [
    ['dam spillway.jpeg', 'green canopy.jpg'],
    ['autumn path.jpg', 'purple ocean mountain.jpg'],
    ['purple ocean mountain.jpg', 'abandoned building.jpg'],
    ['abandoned building.jpg', 'trees shade.jpg'],
    ['trees shade.jpg', 'autumn path.jpg'],
    ['autumn path.jpg', 'green canopy.jpg'],
]

In [ ]:
reference = cv2.imread(selection[0][0])
input = cv2.imread(selection[0][1])
showcase(reference, input)

In [ ]:
reference = cv2.imread(selection[1][0])
input = cv2.imread(selection[1][1])
showcase(reference, input)

In [ ]:
reference = cv2.imread(selection[2][0])
input = cv2.imread(selection[2][1])
showcase(reference, input)

In [ ]:
reference = cv2.imread(selection[3][0])
input = cv2.imread(selection[3][1])
showcase(reference, input)

In [ ]:
reference = cv2.imread(selection[4][0])
input = cv2.imread(selection[4][1])
showcase(reference, input)

In [ ]:
reference = cv2.imread(selection[5][0])
input = cv2.imread(selection[5][1])
showcase(reference, input)

#### References:

1.  [Color transfer between images](https://ieeexplore.ieee.org/abstract/document/946629):
    E. Reinhard, M. Adhikhmin, B. Gooch, P. Shirley.
2.  [Colour Mapping: A Review of Recent Methods, Extensions and Applications](https://onlinelibrary.wiley.com/doi/abs/10.1111/cgf.12671):
    H. Sheikh Faridul, T. Pouli, C. Chamaret et al.
3.  [CIE 15:2004. Colorimetry, 3rd edition](https://cie.co.at/publications/colorimetry-3rd-edition#:~:text=CIE%2015%3A2004%20Colorimetry%20represents,values%2C%20chromaticity%20coordinates%2C%20colour%20spaces)
